In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftshift

def construct_total_dict(root):
    breathhold_dict, full_dict = {}, {}
    for data_type in os.listdir(root):
        subpath = root + data_type + "/"
        for patient_num in os.listdir(subpath):
            patient_path = subpath + patient_num + "/"
            patient_data = []
            for data in os.listdir(patient_path):
                data_path = patient_path + data
                patient_data.append(data_path)
            if "FULL" in data_type:
                full_dict[patient_num] = patient_data
            elif "Breathhold" in data_type:
                breathhold_dict[patient_num] = patient_data
    return breathhold_dict, full_dict

root = './DATA/'

breathhold_dict, full_dict = construct_total_dict(root)

In [2]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return

In [3]:
def extract_data(filepath):
    time_list, amp_list = [], []
    data_type = "Breathhold" if "Breath" in filepath else "Full"
    anterior_count, start_flag = 0, False
    with open(filepath, "r") as file:
        for line in file:
            if start_flag and (line == "\n"): break
            if start_flag:
                time, amplitude = line.strip().split("\t")
                time_list.append(float(time))
                amp_list.append(float(amplitude))
            elif "Anterior-Posterior" in line:
                anterior_count += 1
            if data_type == "Breathhold":
                if (anterior_count > 1) and ("Amplitude" in line): start_flag = True
            elif data_type == "Full":
                if (anterior_count > 0) and ("Amplitude" in line): start_flag = True

    return time_list, amp_list

In [4]:
def extract_beam_moments(filepath):
    time_list, state_list = [], []
    beam_flag, start_flag = False, False
    with open(filepath, "r") as file:
        for line in file:
            if (beam_flag and start_flag) and line == "\n": break
            elif (beam_flag and start_flag):
                time, state = line.strip().split("\t")
                time_list.append(float(time))
                state_list.append(int(state))
            elif "Beam Enable/Disable Moments" in line: beam_flag = True
            elif beam_flag and ("Enable" in line): start_flag = True

    return time_list, state_list

In [5]:
def merge_data(dict, ID):
    path_list = dict[ID]
    merged_time, merged_amp = [], []
    merged_beamtime, merged_state = [], []
    for datapath in path_list:
        period_of_prev_field = 0.0 if ( len(merged_time) == 0 ) else ( merged_time[-1] + 0.015 )
        time, amplitude = extract_data(datapath)
        beamtime, state = extract_beam_moments(datapath)
        merged_time += [x + period_of_prev_field for x in time]
        merged_amp += amplitude
        merged_beamtime += [x + period_of_prev_field for x in beamtime]
        merged_state += state

    return merged_time, merged_amp, merged_beamtime, merged_state

In [6]:
def split_beams(Time, Amp, Beams, States):
    beam_ID = 1
    time_dict, beam_dict = {}, {}
    for state_index in range(len(States)-1):
        curr_state = States[state_index]
        curr_beamtime, next_beamtime = Beams[state_index], Beams[state_index+1]
        if curr_state == 1:
            curr_timelist, curr_beamlist = [], []
            for amp_index in range(len(Amp)):
                if (Time[amp_index] >= curr_beamtime) and (Time[amp_index] <= next_beamtime):
                    curr_timelist.append(Time[amp_index])
                    curr_beamlist.append(Amp[amp_index])
        elif curr_state == 0:
            time_dict[beam_ID] = curr_timelist
            beam_dict[beam_ID] = curr_beamlist
            beam_ID += 1
    return time_dict, beam_dict

In [7]:
def plot_time_series(Times, Amps):
    fft_amp = fftshift(fft(np.array(Amps)))
    N = len(Amps)
    fs = 1/0.015
    freqs = np.linspace(0, N-1, N) * (fs/N)
    fig, ax = plt.subplots(1, 2, figsize=(14, 10))
    ax[0].plot(Times, Amps)
    ax[0].set_title("Time domain data")
    ax[0].set_xlabel("Time (s)"), ax[0].set_ylabel("Amplitude (cm)")
    ax[1].plot(freqs, np.abs(fft_amp))
    ax[1].set_title("Frequency domain data")
    ax[1].set_xlabel("Frequency (Hz)"), ax[0].set_ylabel("Amplitude (cm)")
    plt.close()

    return fig